In [ ]:
from tensorflow import kerastuner as kt
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from collections import deque

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import statistics

sys.path.append("code/.")
from sklearn import datasets
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import average_precision_score
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import StackingClassifier
import eli5

In [ ]:
data = pd.read_csv("data/data_preds.csv")
data.shape

In [ ]:
X = data.drop(columns = 'K')
y = data['K']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 123)

In [ ]:
ord_feats = ['Date']
cat_feats = ['Team_x', 'Opp']
num_feats = data.drop(columns = ['Date', 'Team_x', 'Opp']).columns.tolist()

In [ ]:
ct = make_column_transformer(
    (StandardScaler(), num_feats),
    (OneHotEncoder(), cat_feats), 
    (OrdinalEncoder(), ord_feats))

In [ ]:
transformed_x_train = ct.fit_transform(X_train)
transformed_x_test = ct.transform(X_test)

In [ ]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers',min_value=1,max_value=40)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=300,step=8),activation= hp.Choice('activation' + str(i), values=['relu','tanh','sigmoid']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6])))
    else:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128,step=8),activation= hp.Choice('activation' + str(i), values=['relu','tanh','sigmoid'])))
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6])))
    counter+=1
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
  return model

model.fit(transformed_x_train, y_train, epochs = 100, batch_size = 50)
preds = mod.predict(transformed_x_test)
np.sqrt(mean_squared_error(y_test, preds)
